## Narrative Data

In [10]:
import pandas as pd
import os

In [79]:
narrative_files = pd.DataFrame

for file in os.listdir("data"):
    if file.startswith("NarrativeData_"):
        df = df.append(pd.read_json(os.path.join('data/', file), orient='split'))

In [80]:
df.shape

(81133, 3)

In [81]:
df.head()

,EventId,narrative,probable_cause
0,20070804X01107,NTSB investigators may not have traveled in su...,the pilots failure to maintain clearance with ...
1,20001208X09357,NTSB investigators either traveled in support ...,the pilots failure to maintain adequate airspe...
2,20001212X23867,NTSB investigators may not have traveled in su...,failure of the free turbine intermediate pinio...
3,20001214X37502,NTSB investigators may not have traveled in su...,
4,20001206X02609,NTSB investigators either traveled in support ...,improper loading of the airplane contributing ...


In [15]:
import scipy as sp
import sklearn
import sys
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize
import pickle 
import re

In [16]:
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
tokenizer = word_tokenize
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [82]:
#Clean Dataset

#lowercase
df['probable_cause'] = df['probable_cause'].apply(lambda x: " ".join(x.lower() for x in x.split()))

#remove punctuation
df['probable_cause'] = df['probable_cause'].str.replace('[^\w\s]','')

In [83]:
df_lda = pd.DataFrame(df['probable_cause'])
df_lda.columns = ['probable_cause']
df_lda.head()

,probable_cause
0,the pilots failure to maintain clearance with ...
1,the pilots failure to maintain adequate airspe...
2,failure of the free turbine intermediate pinio...
3,
4,improper loading of the airplane contributing ...


In [84]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
len(stop)

179

In [ ]:
for idx in range(len(df_lda)):
    df_lda.iloc[idx]['probable_cause'] = [ps.stem(word) for word in word_tokenize(df_lda.iloc[idx]['probable_cause'].lower()) if word not in stop]

In [ ]:
df_lda.head()

In [ ]:
training = [value[0] for value in df_lda.iloc[0:].values]
num_topics = 5

In [ ]:
id2word = gensim.corpora.Dictionary(training)
corpus = [id2word.doc2bow(text) for text in training]
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)

In [ ]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 40);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [ ]:
lda_topics = get_lda_topics(lda, num_topics)
lda_topics.head(10)